# **Pre-processing**

---

## **Import Libraries & Data Loading**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os.path as osp

In [2]:
data_folder = osp.join('.', 'dataset')

In [3]:
humidity_df = pd.read_csv(osp.join(data_folder, 'humidity.csv'))
temp_df=pd.read_csv(osp.join(data_folder,'temperature.csv'))
pressure_df = pd.read_csv(osp.join(data_folder, 'pressure.csv'))
wind_direct_df = pd.read_csv(osp.join(data_folder, 'wind_direction.csv'))
wind_speed_df = pd.read_csv(osp.join(data_folder,'wind_speed.csv'))

In [4]:
weather_descript_df = pd.read_csv(osp.join(data_folder,'weather_description.csv'))

In [5]:
city_attri_df = pd.read_csv(osp.join(data_folder,'city_attributes.csv'))

## **Humidity**

In [6]:
humidity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          43427 non-null  float64
 2   Portland           44804 non-null  float64
 3   San Francisco      44311 non-null  float64
 4   Seattle            44964 non-null  float64
 5   Los Angeles        45101 non-null  float64
 6   San Diego          44909 non-null  float64
 7   Las Vegas          44411 non-null  float64
 8   Phoenix            43945 non-null  float64
 9   Albuquerque        44543 non-null  float64
 10  Denver             43445 non-null  float64
 11  San Antonio        44689 non-null  float64
 12  Dallas             44934 non-null  float64
 13  Houston            45132 non-null  float64
 14  Kansas City        44741 non-null  float64
 15  Minneapolis        44743 non-null  float64
 16  Saint Louis        439

In [7]:
#date & time
humidity_df['datetime'] = pd.to_datetime(humidity_df['datetime'])
humidity_df['date'] = humidity_df['datetime'].dt.date
humidity_df['time'] = humidity_df['datetime'].dt.time

humidity_df.drop(columns=['datetime','time'], inplace=True)

cols = ['date'] + [col for col in humidity_df.columns if col not in ['date']]
humidity_df = humidity_df[cols]


In [8]:
humidity_df.head()

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN
1,2012-10-01,76.0,81.0,88.0,81.0,88.0,82.0,22.0,23.0,50.0,...,71.0,58.0,93.0,68.0,50.0,63.0,22.0,51.0,51.0,50.0
2,2012-10-01,76.0,80.0,87.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,91.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
3,2012-10-01,76.0,80.0,86.0,80.0,88.0,81.0,21.0,23.0,49.0,...,70.0,57.0,87.0,68.0,51.0,62.0,22.0,51.0,51.0,50.0
4,2012-10-01,77.0,80.0,85.0,79.0,88.0,81.0,21.0,23.0,49.0,...,69.0,57.0,84.0,68.0,52.0,62.0,22.0,51.0,51.0,50.0


In [9]:
#fill missing values
humidity_df = humidity_df.iloc[1:]
humidity_df = humidity_df.ffill() #forward fill
humidity_df.tail()

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
45248,2017-11-29,87.0,81.0,22.0,93.0,24.0,72.0,18.0,68.0,37.0,...,27.0,58.0,64.0,37.0,57.0,60.0,100.0,96.0,96.0,60.0
45249,2017-11-29,87.0,71.0,22.0,87.0,21.0,72.0,18.0,73.0,34.0,...,29.0,58.0,59.0,74.0,57.0,60.0,100.0,96.0,96.0,60.0
45250,2017-11-29,87.0,71.0,22.0,93.0,23.0,68.0,17.0,60.0,32.0,...,31.0,58.0,66.0,74.0,57.0,60.0,100.0,96.0,96.0,60.0
45251,2017-11-29,87.0,71.0,22.0,87.0,14.0,63.0,17.0,33.0,30.0,...,26.0,58.0,58.0,56.0,57.0,60.0,100.0,96.0,96.0,60.0
45252,2017-11-30,87.0,76.0,22.0,75.0,56.0,72.0,17.0,23.0,34.0,...,32.0,58.0,58.0,56.0,57.0,60.0,100.0,96.0,96.0,60.0


In [10]:
humidity_df = humidity_df.groupby(humidity_df['date']).mean(numeric_only=True).reset_index()

In [11]:
humidity_df.tail()

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
1882,2017-11-26,87.0,87.625000,22.0,93.291667,86.666667,88.250000,38.166667,43.208333,29.750000,...,53.125000,58.0,78.166667,54.666667,57.0,60.0,100.0,96.0,96.0,60.0
1883,2017-11-27,87.0,92.875000,22.0,85.416667,77.333333,81.333333,34.458333,34.041667,29.041667,...,49.375000,58.0,74.125000,55.958333,57.0,60.0,100.0,96.0,96.0,60.0
1884,2017-11-28,87.0,88.458333,22.0,85.750000,54.958333,76.375000,33.125000,46.208333,31.458333,...,54.666667,58.0,64.833333,57.541667,57.0,60.0,100.0,96.0,96.0,60.0
1885,2017-11-29,87.0,85.500000,22.0,92.041667,58.458333,75.125000,25.500000,58.333333,52.000000,...,65.375000,58.0,75.375000,68.666667,57.0,60.0,100.0,96.0,96.0,60.0
1886,2017-11-30,87.0,76.000000,22.0,75.000000,56.000000,72.000000,17.000000,23.000000,34.000000,...,32.000000,58.0,58.000000,56.000000,57.0,60.0,100.0,96.0,96.0,60.0


## **Pressure**

In [13]:
pressure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          41019 non-null  float64
 2   Portland           45249 non-null  float64
 3   San Francisco      44438 non-null  float64
 4   Seattle            45240 non-null  float64
 5   Los Angeles        45001 non-null  float64
 6   San Diego          45078 non-null  float64
 7   Las Vegas          45165 non-null  float64
 8   Phoenix            44659 non-null  float64
 9   Albuquerque        44797 non-null  float64
 10  Denver             44710 non-null  float64
 11  San Antonio        45236 non-null  float64
 12  Dallas             45193 non-null  float64
 13  Houston            45244 non-null  float64
 14  Kansas City        45132 non-null  float64
 15  Minneapolis        45236 non-null  float64
 16  Saint Louis        451

In [14]:
#date & time
pressure_df['datetime'] = pd.to_datetime(pressure_df['datetime'])
pressure_df['date'] = pressure_df['datetime'].dt.date
pressure_df['time'] = pressure_df['datetime'].dt.time

pressure_df.drop(columns=['datetime'], inplace=True)

cols = ['date', 'time'] + [col for col in pressure_df.columns if col not in ['date', 'time']]
pressure_df = pressure_df[cols]


In [15]:
#fill missing values
pressure_df = pressure_df.iloc[1:]
pressure_df = pressure_df.ffill() #forward fill
pressure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 1 to 45252
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               45252 non-null  object 
 1   time               45252 non-null  object 
 2   Vancouver          45232 non-null  float64
 3   Portland           45252 non-null  float64
 4   San Francisco      45252 non-null  float64
 5   Seattle            45252 non-null  float64
 6   Los Angeles        45252 non-null  float64
 7   San Diego          45252 non-null  float64
 8   Las Vegas          45252 non-null  float64
 9   Phoenix            45252 non-null  float64
 10  Albuquerque        45252 non-null  float64
 11  Denver             45252 non-null  float64
 12  San Antonio        45252 non-null  float64
 13  Dallas             45252 non-null  float64
 14  Houston            45252 non-null  float64
 15  Kansas City        45252 non-null  float64
 16  Minneapolis        452

In [16]:
pressure_df = pressure_df.groupby(pressure_df['date']).mean(numeric_only=True).reset_index()
pressure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1886 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Temperature**

In [18]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          44458 non-null  float64
 2   Portland           45252 non-null  float64
 3   San Francisco      44460 non-null  float64
 4   Seattle            45250 non-null  float64
 5   Los Angeles        45250 non-null  float64
 6   San Diego          45252 non-null  float64
 7   Las Vegas          45252 non-null  float64
 8   Phoenix            45250 non-null  float64
 9   Albuquerque        45252 non-null  float64
 10  Denver             45252 non-null  float64
 11  San Antonio        45252 non-null  float64
 12  Dallas             45249 non-null  float64
 13  Houston            45250 non-null  float64
 14  Kansas City        45252 non-null  float64
 15  Minneapolis        45240 non-null  float64
 16  Saint Louis        452

In [19]:
#date & time
temp_df['datetime'] = pd.to_datetime(temp_df['datetime'])
temp_df['date'] = temp_df['datetime'].dt.date
temp_df['time'] = temp_df['datetime'].dt.time

temp_df.drop(columns=['datetime'], inplace=True)

cols = ['date', 'time'] + [col for col in temp_df.columns if col not in ['date', 'time']]
temp_df = temp_df[cols]


In [20]:
#fill missing values
temp_df = temp_df.iloc[1:]
temp_df = temp_df.ffill() #forward fill
temp_df.tail()

,date,time,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
45248,2017-11-29,20:00:00,288.01,282.00,302.07,280.82,293.55,292.15,289.54,294.71,...,290.24,284.98,275.13,288.08,285.944,294.15,291.019,299.369,299.369,294.15
45249,2017-11-29,21:00:00,288.01,282.89,302.07,281.65,295.68,292.74,290.61,295.59,...,289.24,284.98,274.13,286.02,285.944,294.15,291.019,299.369,299.369,294.15
45250,2017-11-29,22:00:00,288.01,283.39,302.07,282.75,295.96,292.58,291.34,296.25,...,286.78,284.98,273.48,283.94,285.944,294.15,291.019,299.369,299.369,294.15
45251,2017-11-29,23:00:00,288.01,283.02,302.07,282.96,295.65,292.61,292.15,297.15,...,284.57,284.98,272.48,282.17,285.944,294.15,291.019,299.369,299.369,294.15
45252,2017-11-30,00:00:00,288.01,282.28,302.07,283.04,294.93,291.40,291.64,297.15,...,283.42,284.98,271.80,280.65,285.944,294.15,291.019,299.369,299.369,294.15


In [21]:
temp_df = temp_df.groupby(temp_df['date']).mean(numeric_only=True).reset_index()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Wind Direction**

In [26]:
wind_direct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 1 to 45252
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               45252 non-null  object 
 1   time               45252 non-null  object 
 2   Vancouver          45252 non-null  float64
 3   Portland           45252 non-null  float64
 4   San Francisco      45252 non-null  float64
 5   Seattle            45252 non-null  float64
 6   Los Angeles        45252 non-null  float64
 7   San Diego          45252 non-null  float64
 8   Las Vegas          45252 non-null  float64
 9   Phoenix            45252 non-null  float64
 10  Albuquerque        45252 non-null  float64
 11  Denver             45252 non-null  float64
 12  San Antonio        45252 non-null  float64
 13  Dallas             45252 non-null  float64
 14  Houston            45252 non-null  float64
 15  Kansas City        45252 non-null  float64
 16  Minneapolis        452

In [23]:
#date & time
wind_direct_df['datetime'] = pd.to_datetime(wind_direct_df['datetime'])
wind_direct_df['date'] = wind_direct_df['datetime'].dt.date
wind_direct_df['time'] = wind_direct_df['datetime'].dt.time

wind_direct_df.drop(columns=['datetime'], inplace=True)

cols = ['date', 'time'] + [col for col in wind_direct_df.columns if col not in ['date', 'time']]
wind_direct_df = wind_direct_df[cols]


In [24]:
#fill missing values
wind_direct_df = wind_direct_df.iloc[1:]
wind_direct_df = wind_direct_df.ffill() #forward fill
wind_direct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 1 to 45252
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               45252 non-null  object 
 1   time               45252 non-null  object 
 2   Vancouver          45252 non-null  float64
 3   Portland           45252 non-null  float64
 4   San Francisco      45252 non-null  float64
 5   Seattle            45252 non-null  float64
 6   Los Angeles        45252 non-null  float64
 7   San Diego          45252 non-null  float64
 8   Las Vegas          45252 non-null  float64
 9   Phoenix            45252 non-null  float64
 10  Albuquerque        45252 non-null  float64
 11  Denver             45252 non-null  float64
 12  San Antonio        45252 non-null  float64
 13  Dallas             45252 non-null  float64
 14  Houston            45252 non-null  float64
 15  Kansas City        45252 non-null  float64
 16  Minneapolis        452

In [27]:
wind_direct_df = wind_direct_df.groupby(wind_direct_df['date']).mean(numeric_only=True).reset_index()
wind_direct_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Wind Speed**

In [28]:
wind_speed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45253 entries, 0 to 45252
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   datetime           45253 non-null  object 
 1   Vancouver          44458 non-null  float64
 2   Portland           45252 non-null  float64
 3   San Francisco      44459 non-null  float64
 4   Seattle            45252 non-null  float64
 5   Los Angeles        45252 non-null  float64
 6   San Diego          45252 non-null  float64
 7   Las Vegas          45238 non-null  float64
 8   Phoenix            45251 non-null  float64
 9   Albuquerque        45249 non-null  float64
 10  Denver             45251 non-null  float64
 11  San Antonio        45252 non-null  float64
 12  Dallas             45251 non-null  float64
 13  Houston            45250 non-null  float64
 14  Kansas City        45252 non-null  float64
 15  Minneapolis        45250 non-null  float64
 16  Saint Louis        452

In [29]:
#date & time
wind_speed_df['datetime'] = pd.to_datetime(wind_speed_df['datetime'])
wind_speed_df['date'] = wind_speed_df['datetime'].dt.date
wind_speed_df['time'] = wind_speed_df['datetime'].dt.time

wind_speed_df.drop(columns=['datetime'], inplace=True)

cols = ['date', 'time'] + [col for col in wind_speed_df.columns if col not in ['date', 'time']]
wind_speed_df = wind_speed_df[cols]


In [30]:
#fill missing values
wind_speed_df = wind_speed_df.iloc[1:]
wind_speed_df = wind_speed_df.ffill() #forward fill
wind_speed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45252 entries, 1 to 45252
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               45252 non-null  object 
 1   time               45252 non-null  object 
 2   Vancouver          45252 non-null  float64
 3   Portland           45252 non-null  float64
 4   San Francisco      45252 non-null  float64
 5   Seattle            45252 non-null  float64
 6   Los Angeles        45252 non-null  float64
 7   San Diego          45252 non-null  float64
 8   Las Vegas          45252 non-null  float64
 9   Phoenix            45252 non-null  float64
 10  Albuquerque        45252 non-null  float64
 11  Denver             45252 non-null  float64
 12  San Antonio        45252 non-null  float64
 13  Dallas             45252 non-null  float64
 14  Houston            45252 non-null  float64
 15  Kansas City        45252 non-null  float64
 16  Minneapolis        452

In [31]:
wind_speed_df = wind_speed_df.groupby(wind_speed_df['date']).mean(numeric_only=True).reset_index()
wind_speed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1887 non-null   object 
 1   Vancouver          1887 non-null   float64
 2   Portland           1887 non-null   float64
 3   San Francisco      1887 non-null   float64
 4   Seattle            1887 non-null   float64
 5   Los Angeles        1887 non-null   float64
 6   San Diego          1887 non-null   float64
 7   Las Vegas          1887 non-null   float64
 8   Phoenix            1887 non-null   float64
 9   Albuquerque        1887 non-null   float64
 10  Denver             1887 non-null   float64
 11  San Antonio        1887 non-null   float64
 12  Dallas             1887 non-null   float64
 13  Houston            1887 non-null   float64
 14  Kansas City        1887 non-null   float64
 15  Minneapolis        1887 non-null   float64
 16  Saint Louis        1887 

## **Save preprocessed data**

In [33]:
humidity_df.to_csv(osp.join(data_folder, 'humidity_preprocessed.csv'), index=False)
pressure_df.to_csv(osp.join(data_folder, 'pressure_preprocessed.csv'), index=False)
temp_df.to_csv(osp.join(data_folder, 'temp_preprocessed.csv'), index=False)
wind_direct_df.to_csv(osp.join(data_folder, 'wind_direct_preprocessed.csv'), index=False)
wind_speed_df.to_csv(osp.join(data_folder, 'wind_speed_preprocessed.csv'), index=False)
